In [30]:
import pandas as pd
import cal_service as cal
import db_service as db
import datetime

dtStr = '2019-01-01'

dt = datetime.datetime.strptime(dtStr, '%Y-%m-%d').date()
dtPre = dt - datetime.timedelta(days=1)
dtPreStr = datetime.datetime.strftime(dtPre, '%Y-%m-%d')

db = sqlite3.connect('AMS.db')
query = "select * from HOlding_Table_Test where Date = '{}'".format(dtPreStr)
dfHoldPre = pd.read_sql(query, con=db)
dfMVPre = dfHoldPre[['AccountID','SymbolCode','SymbolName','CurTrade','Qty','Cost','Close']]
dfMVPre.rename(columns={'Qty':'QtyPre', 'Cost':'CostPre', 'Close':'ClosePre'}, inplace = True)
dfMVPre.insert(0,'Date',dt)
dfMVPre

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre
0,2019-01-01,CITIC,510500.SS,500ETF,CNY,5000.00,4.870,4.451
1,2019-01-01,CITIC,600177.SS,雅戈尔,CNY,2000.00,8.593,7.190
2,2019-01-01,CITIC,601088.SS,中国神华,CNY,2600.00,11.275,17.960
3,2019-01-01,CITIC,000726.SZ,鲁泰A,CNY,4000.00,11.174,9.710
4,2019-01-01,CITIC,CNY,人民币,CNY,61575.92,1.000,1.000
5,2019-01-01,CMS,512880.SS,证券ETF,CNY,40000.00,0.741,0.717
6,2019-01-01,CMS,600030.SS,中信证券,CNY,1000.00,19.136,16.010
7,2019-01-01,CMS,601166.SS,兴业银行,CNY,2800.00,15.892,14.940
8,2019-01-01,CMS,CNY,人民币,CNY,45255.81,1.000,1.000
9,2019-01-01,CMS,200625.SZ,长安B,HKD,5500.00,7.979,3.490


In [31]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,Qty AS QtyBuy,SettleAmt from order_table_test where OrderType in ('Buy','Bonus') and Date = '{}'".format(dt)
dfBuy =  pd.read_sql(query, con=db)
dfBuy = dfBuy.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
if dfBuy.empty == False:
    dfBuy.insert(0,'Date',dt)
    dfBuy.loc[:,'CostBuy'] = abs(dfBuy['SettleAmt'])/dfBuy['QtyBuy']
    dfBuy.drop(columns=['SettleAmt'],inplace=True)
    dfBuy.reset_index(inplace =True)
    df_MV_Buy = pd.merge(dfMVPre,dfBuy,on=['Date','AccountID','SymbolCode','SymbolName','CurTrade'],how='outer') 
else:
    df_MV_Buy = dfMVPre
    df_MV_Buy.loc[:,'QtyBuy'] = 0
    df_MV_Buy.loc[:,'CostBuy'] = 0

df_MV_Buy.fillna(0,inplace=True)
df_MV_Buy

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre,QtyBuy,CostBuy
0,2019-01-01,CITIC,510500.SS,500ETF,CNY,5000.00,4.870,4.451,0.0,0.00000
1,2019-01-01,CITIC,600177.SS,雅戈尔,CNY,2000.00,8.593,7.190,0.0,0.00000
2,2019-01-01,CITIC,601088.SS,中国神华,CNY,2600.00,11.275,17.960,0.0,0.00000
3,2019-01-01,CITIC,000726.SZ,鲁泰A,CNY,4000.00,11.174,9.710,0.0,0.00000
4,2019-01-01,CITIC,CNY,人民币,CNY,61575.92,1.000,1.000,0.0,0.00000
5,2019-01-01,CMS,512880.SS,证券ETF,CNY,40000.00,0.741,0.717,0.0,0.00000
6,2019-01-01,CMS,600030.SS,中信证券,CNY,1000.00,19.136,16.010,0.0,0.00000
7,2019-01-01,CMS,601166.SS,兴业银行,CNY,2800.00,15.892,14.940,0.0,0.00000
8,2019-01-01,CMS,CNY,人民币,CNY,45255.81,1.000,1.000,0.0,0.00000
9,2019-01-01,CMS,200625.SZ,长安B,HKD,5500.00,7.979,3.490,6400.0,4.12412


In [32]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,Qty AS QtySell,SettleAmt from order_table_test where OrderType='Sell' and Date = '{}'".format(dt)
dfSell =  pd.read_sql(query, con=db)
dfSell = dfSell.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
if dfSell.empty == False:
    dfSell.insert(0,'Date',dt)
    dfSell[:,'PriceSell'] = abs(dfSell['SettleAmt'])/dfSell['QtySell']
    dfSell.drop(columns=['SettleAmt'],inplace=True)
    dfSell.reset_index(inplace =True)
    df_MV_Buy_Sell = pd.merge(df_MV_Buy,dfSell,on=['Date','AccountID','SymbolCode','SymbolName','CurTrade'],how='outer')
else:
    df_MV_Buy_Sell = df_MV_Buy
    df_MV_Buy_Sell.loc[:,'QtySell'] = 0
    df_MV_Buy_Sell.loc[:,'PriceSell'] = 0

df_MV_Buy_Sell.fillna(0,inplace=True)
df_MV_Buy_Sell

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre,QtyBuy,CostBuy,QtySell,PriceSell
0,2019-01-01,CITIC,510500.SS,500ETF,CNY,5000.00,4.870,4.451,0.0,0.00000,0,0
1,2019-01-01,CITIC,600177.SS,雅戈尔,CNY,2000.00,8.593,7.190,0.0,0.00000,0,0
2,2019-01-01,CITIC,601088.SS,中国神华,CNY,2600.00,11.275,17.960,0.0,0.00000,0,0
3,2019-01-01,CITIC,000726.SZ,鲁泰A,CNY,4000.00,11.174,9.710,0.0,0.00000,0,0
4,2019-01-01,CITIC,CNY,人民币,CNY,61575.92,1.000,1.000,0.0,0.00000,0,0
5,2019-01-01,CMS,512880.SS,证券ETF,CNY,40000.00,0.741,0.717,0.0,0.00000,0,0
6,2019-01-01,CMS,600030.SS,中信证券,CNY,1000.00,19.136,16.010,0.0,0.00000,0,0
7,2019-01-01,CMS,601166.SS,兴业银行,CNY,2800.00,15.892,14.940,0.0,0.00000,0,0
8,2019-01-01,CMS,CNY,人民币,CNY,45255.81,1.000,1.000,0.0,0.00000,0,0
9,2019-01-01,CMS,200625.SZ,长安B,HKD,5500.00,7.979,3.490,6400.0,4.12412,0,0


In [33]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,SettleAmt AS Interest from order_table_test where OrderType ='Interest' and Date = '{}'".format(dt)
dfInterest =  pd.read_sql(query, con=db)
dfInterest = dfInterest.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
if dfInterest.empty == False:
    dfInterest.insert(0,'Date',dt)
    dfInterest.reset_index(inplace =True)
    df_MV_Buy_Sell_Interest = pd.merge(df_MV_Buy_Sell,dfInterest,on=['Date','AccountID','SymbolCode','SymbolName','CurTrade'],how='outer')
else:
    df_MV_Buy_Sell_Interest = df_MV_Buy_Sell
    df_MV_Buy_Sell_Interest.loc[:,'Interest'] = 0

df_MV_Buy_Sell_Interest.fillna(0,inplace=True)
df_MV_Buy_Sell_Interest

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre,QtyBuy,CostBuy,QtySell,PriceSell,Interest
0,2019-01-01,CITIC,510500.SS,500ETF,CNY,5000.00,4.870,4.451,0.0,0.00000,0,0,0.00
1,2019-01-01,CITIC,600177.SS,雅戈尔,CNY,2000.00,8.593,7.190,0.0,0.00000,0,0,0.00
2,2019-01-01,CITIC,601088.SS,中国神华,CNY,2600.00,11.275,17.960,0.0,0.00000,0,0,0.00
3,2019-01-01,CITIC,000726.SZ,鲁泰A,CNY,4000.00,11.174,9.710,0.0,0.00000,0,0,0.00
4,2019-01-01,CITIC,CNY,人民币,CNY,61575.92,1.000,1.000,0.0,0.00000,0,0,8.91
5,2019-01-01,CMS,512880.SS,证券ETF,CNY,40000.00,0.741,0.717,0.0,0.00000,0,0,0.00
6,2019-01-01,CMS,600030.SS,中信证券,CNY,1000.00,19.136,16.010,0.0,0.00000,0,0,0.00
7,2019-01-01,CMS,601166.SS,兴业银行,CNY,2800.00,15.892,14.940,0.0,0.00000,0,0,0.00
8,2019-01-01,CMS,CNY,人民币,CNY,45255.81,1.000,1.000,0.0,0.00000,0,0,0.00
9,2019-01-01,CMS,200625.SZ,长安B,HKD,5500.00,7.979,3.490,6400.0,4.12412,0,0,0.00


In [34]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,SettleAmt AS MoneyOut from order_table_test where OrderType ='Withdraw' and Date = '{}'".format(dt)
dfMoneyOut =  pd.read_sql(query, con=db)
dfMoneyOut = dfMoneyOut.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
dfMoneyOut

,,,,MoneyOut
AccountID,SymbolCode,SymbolName,CurTrade,
CMS,CNY,人民币,CNY,-40000


In [35]:
query = "select AccountID,SymbolCode,SymbolName,CurTrade,SettleAmt AS MoneyIN from order_table_test where OrderType ='Deposit' and Date = '{}'".format(dt)
dfMoneyIn =  pd.read_sql(query, con=db)
dfMoneyIn = dfMoneyIn.groupby(['AccountID','SymbolCode','SymbolName','CurTrade']).sum()
dfMoneyIn

,,,,MoneyIN
AccountID,SymbolCode,SymbolName,CurTrade,
CMS,HKD,港币,HKD,23300


In [36]:
dfMoney = pd.merge(dfMoneyIn,dfMoneyOut,on=['AccountID','SymbolCode','SymbolName','CurTrade'],how='outer')
dfMoney.fillna(0,inplace=True)
dfMoney.reset_index(inplace=True)
dfMoney.insert(0,'Date',dt)
dfMoney

,Date,AccountID,SymbolCode,SymbolName,CurTrade,MoneyIN,MoneyOut
0,2019-01-01,CMS,HKD,港币,HKD,23300.0,0.0
1,2019-01-01,CMS,CNY,人民币,CNY,0.0,-40000.0


In [37]:
dfAll = pd.merge(df_MV_Buy_Sell_Interest,dfMoney, on=['Date','AccountID','SymbolCode','SymbolName','CurTrade'],how='outer')
dfAll.fillna(0,inplace=True)
dfAll

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre,QtyBuy,CostBuy,QtySell,PriceSell,Interest,MoneyIN,MoneyOut
0,2019-01-01,CITIC,510500.SS,500ETF,CNY,5000.00,4.870,4.451,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
1,2019-01-01,CITIC,600177.SS,雅戈尔,CNY,2000.00,8.593,7.190,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
2,2019-01-01,CITIC,601088.SS,中国神华,CNY,2600.00,11.275,17.960,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
3,2019-01-01,CITIC,000726.SZ,鲁泰A,CNY,4000.00,11.174,9.710,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
4,2019-01-01,CITIC,CNY,人民币,CNY,61575.92,1.000,1.000,0.0,0.00000,0.0,0.0,8.91,0.0,0.0
5,2019-01-01,CMS,512880.SS,证券ETF,CNY,40000.00,0.741,0.717,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
6,2019-01-01,CMS,600030.SS,中信证券,CNY,1000.00,19.136,16.010,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
7,2019-01-01,CMS,601166.SS,兴业银行,CNY,2800.00,15.892,14.940,0.0,0.00000,0.0,0.0,0.00,0.0,0.0
8,2019-01-01,CMS,CNY,人民币,CNY,45255.81,1.000,1.000,0.0,0.00000,0.0,0.0,0.00,0.0,-40000.0
9,2019-01-01,CMS,200625.SZ,长安B,HKD,5500.00,7.979,3.490,6400.0,4.12412,0.0,0.0,0.00,0.0,0.0


In [38]:
dfAll.loc[:,'Qty'] = dfAll.apply(lambda x: x['QtyPre']+x['MoneyIN']+x['MoneyOut']+x['Interest'] if x['SymbolCode'] in ['HKD','CNY'] else x['QtyPre']+x['QtyBuy']-x['QtySell'],axis=1)
dfAll.loc[:,'Cost'] = dfAll.apply(lambda x: 0 if x['QtyPre']+x['QtyBuy']==0 else (x['QtyPre']*x['CostPre'] + x['QtyBuy']*x['CostBuy']-x['Interest'])/(x['QtyPre']+x['QtyBuy']),axis=1)
dfAll.loc[:,'Cost'] = dfAll.apply(lambda x: 1.0 if x['SymbolCode'] in ['HKD','CNY'] else x['Cost'], axis=1 )
dfAll

,Date,AccountID,SymbolCode,SymbolName,CurTrade,QtyPre,CostPre,ClosePre,QtyBuy,CostBuy,QtySell,PriceSell,Interest,MoneyIN,MoneyOut,Qty,Cost
0,2019-01-01,CITIC,510500.SS,500ETF,CNY,5000.00,4.870,4.451,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,5000.00,4.870000
1,2019-01-01,CITIC,600177.SS,雅戈尔,CNY,2000.00,8.593,7.190,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,2000.00,8.593000
2,2019-01-01,CITIC,601088.SS,中国神华,CNY,2600.00,11.275,17.960,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,2600.00,11.275000
3,2019-01-01,CITIC,000726.SZ,鲁泰A,CNY,4000.00,11.174,9.710,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,4000.00,11.174000
4,2019-01-01,CITIC,CNY,人民币,CNY,61575.92,1.000,1.000,0.0,0.00000,0.0,0.0,8.91,0.0,0.0,61584.83,1.000000
5,2019-01-01,CMS,512880.SS,证券ETF,CNY,40000.00,0.741,0.717,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,40000.00,0.741000
6,2019-01-01,CMS,600030.SS,中信证券,CNY,1000.00,19.136,16.010,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,1000.00,19.136000
7,2019-01-01,CMS,601166.SS,兴业银行,CNY,2800.00,15.892,14.940,0.0,0.00000,0.0,0.0,0.00,0.0,0.0,2800.00,15.892000
8,2019-01-01,CMS,CNY,人民币,CNY,45255.81,1.000,1.000,0.0,0.00000,0.0,0.0,0.00,0.0,-40000.0,5255.81,1.000000
9,2019-01-01,CMS,200625.SZ,长安B,HKD,5500.00,7.979,3.490,6400.0,4.12412,0.0,0.0,0.00,0.0,0.0,11900.00,5.905787


In [39]:
import pandas_datareader.data as web
import datetime

symbol_list = list(set(list(dfAll['SymbolCode'])))
if 'CNY' in symbol_list:
    symbol_list.remove('CNY')
if 'HKD' in symbol_list:
    symbol_list.remove('HKD')
if 'USD' in symbol_list:
    symbol_list.remove('USD')

symbol_list


['000726.SZ',
 '200625.SZ',
 '601088.SS',
 '512880.SS',
 '600030.SS',
 '601166.SS',
 '510500.SS',
 '600177.SS']

In [54]:
import time
print(dt)
symbol_list = ['000726.SZ',
 '200625.SZ',
 '601088.SS']
price_dict = {}
for code in symbol_list:
    close = web.get_data_yahoo(code,dt,dt)
    #time.sleep(1)
    print(close)
    price_dict[code] = close['Close']

stocks = pd.DataFrame(price_dict)
stocks.T

2019-01-01
            High  Low  Open  Close   Volume  Adj Close
Date                                                  
2019-01-02  9.77  9.5  9.72   9.63  2145017    9.16343
            High   Low  Open  Close   Volume  Adj Close
Date                                                   
2019-01-02  3.56  3.48  3.51    3.5  1761500    3.47776
                 High        Low   Open  Close    Volume  Adj Close
Date                                                               
2019-01-02  18.059999  17.610001  17.93  17.77  14300997  17.011261


Date,2019-01-02 00:00:00
000726.SZ,9.63
200625.SZ,3.50
601088.SS,17.77
